In [ ]:
import pandas as pd


print("Reading pheno")
pheno = pd.read_csv("/content/drive/MyDrive/thesis/Mutual Info Results/HGDP-Pak.txt", delimiter='\t')
pheno.drop(['Sex', 'Superpopulation name', 'Population elastic ID', 'Data collections'], axis=1, inplace=True)

print("Entering Loop")
for chr in ['Y']:
    print("\nReading CSV of Chromosome", chr)
    df = pd.read_csv("/content/drive/MyDrive/thesis/HGDP Chromosomes Results/HGDP-chr"+str(chr)+"-GT-Samples_split&ldpruned&T.csv")
    df.fillna(3, inplace=True)
    df.rename(columns={'Unnamed: 0': 's'}, inplace=True)

    print("Merging Geno Pheno\n")
    merged = pd.merge(left=df, right=pheno, how='left', left_on='s', right_on='Sample name')
    merged.to_csv("/content/drive/MyDrive/thesis/Thesis2-ML/thesis2_chr-"+str(chr)+".csv")

    mi_y = pd.read_csv('/content/drive/MyDrive/thesis/Mutual Info Results/hgdp-'+str(chr)+'_mi_scores.csv', nrows=300)
    bps_mi_y = mi_y['Columns'].to_list()
    bps_mi_y.append('Population name')

    merged1 = merged[[c for c in merged.columns if c in bps_mi_y]]
    print("Exporting")
    merged1.to_csv("/content/drive/MyDrive/thesis/Thesis2-ML/thesis2_chr-"+str(chr)+"_mlready.csv")


Reading pheno
Entering Loop

Reading CSV of Chromosome Y
Merging Geno Pheno

Exporting


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/thesis/thesis2_chr-y_mlready.csv")
df.head()

,Unnamed: 0,"chrY:2829914:[""G"",""A""]","chrY:2840370:[""CGA"",""C""]","chrY:2939022:[""TAA"",""T""]","chrY:2945570:[""A"",""G""]","chrY:2953418:[""T"",""G""]","chrY:2954699:[""A"",""AAAAT""]","chrY:2997466:[""C"",""A""]","chrY:3020162:[""C"",""G""]","chrY:3213771:[""C"",""CA""]",...,"chrY:26532494:[""T"",""C""]","chrY:26559274:[""C"",""T""]","chrY:26641644:[""T"",""G""]","chrY:26650069:[""T"",""G""]","chrY:56735722:[""C"",""T""]","chrY:56759946:[""G"",""T""]","chrY:56835906:[""C"",""CTCCACTCCACTCAAT""]","chrY:56882990:[""G"",""C""]","chrY:56883505:[""G"",""A""]",Population name
0,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,3.0,3.0,3.0,3.0,3.0,Brahui
1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,3.0,3.0,3.0,3.0,Brahui
2,2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,3.0,0.0,0.0,3.0,3.0,3.0,Brahui
3,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,1.0,3.0,3.0,3.0,3.0,Brahui
4,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,1.0,0.0,0.0,0.0,3.0,0.0,3.0,0.0,3.0,Brahui


In [ ]:
import pandas as pd
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from pycaret.classification import *

# for i in [10,20,30,40,50,70,100,150,200,250,300]:

df = pd.read_csv("/content/drive/MyDrive/thesis/thesis2_chr-X_mlready.csv")
df.replace(0, 4, inplace=True)


X_new = SelectKBest(chi2, k=250).fit_transform(df.drop(['Unnamed: 0', 'Population name'],
                                                    axis=1), df['Population name'])

dfnew = pd.DataFrame(X_new)
dfnew['Population name'] = df['Population name']

In [ ]:

exp_mclf101 = setup(data = dfnew, target = 'Population name',
                    session_id=123, normalize=True, fix_imbalance = True)
best = compare_models()


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
nb,Naive Bayes,0.4209,0.6819,0.4312,0.3300,0.3428,0.3330,0.3571,0.050
et,Extra Trees Classifier,0.4009,0.7574,0.3812,0.3681,0.3624,0.3094,0.3225,0.483
lr,Logistic Regression,0.3845,0.7779,0.3938,0.3192,0.3287,0.2909,0.3070,0.429
lda,Linear Discriminant Analysis,0.3745,0.7037,0.3812,0.3898,0.3619,0.2868,0.2981,0.052
knn,K Neighbors Classifier,0.3636,0.7264,0.3688,0.3677,0.3365,0.2751,0.2992,0.172
svm,SVM - Linear Kernel,0.3573,0.0000,0.3750,0.3170,0.3112,0.2635,0.2799,0.093
dt,Decision Tree Classifier,0.3555,0.6332,0.3562,0.3307,0.3233,0.2640,0.2812,0.055
rf,Random Forest Classifier,0.3455,0.7553,0.3500,0.3221,0.3145,0.2501,0.2621,0.496
lightgbm,Light Gradient Boosting Machine,0.3282,0.6624,0.3188,0.3018,0.3017,0.2207,0.2273,0.211
gbc,Gradient Boosting Classifier,0.3273,0.7137,0.3188,0.2816,0.2889,0.2203,0.2294,0.868


INFO:logs:create_model_container: 14
INFO:logs:master_model_container: 14
INFO:logs:display_container: 2
INFO:logs:GaussianNB(priors=None, var_smoothing=1e-09)
INFO:logs:compare_models() succesfully completed......................................


In [ ]:
predict_model(best);

INFO:logs:Initializing predict_model()
INFO:logs:predict_model(estimator=GaussianNB(priors=None, var_smoothing=1e-09), probability_threshold=None, encoded_labels=False, drift_report=False, raw_score=False, round=4, verbose=True, ml_usecase=MLUsecase.CLASSIFICATION, display=None, drift_kwargs=None)
INFO:logs:Checking exceptions
INFO:logs:Preloading libraries
INFO:logs:Preparing display monitor


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Naive Bayes,0.4565,0.6951,0.4673,0.5193,0.4379,0.373,0.3796
